# Dataset Cleaning

# Dataset info
ID: Unique identifier for each entry in the dataset.

Customer_ID: Identifier for each customer.

Month: Month of data collection.

Name: Name of the customer.

Age: Age of the customer.

SSN: Social Security Number of the customer.

Occupation: Occupation of the customer.

Annual_Income: Annual income of the customer.

Monthly_Inhand_Salary: Monthly salary after deductions.

Num_Bank_Accounts: Number of bank accounts the customer has.

Num_Credit_Card: Number of credit cards the customer has.

Interest_Rate: Interest rate applied on loans.

Num_of_Loan: Number of loans the customer has.

Type_of_Loan: Type of loan taken by the customer.

Delay_from_due_date: Number of days delayed from due date for payments.

Num_of_Delayed_Payment: Number of delayed payments made by the customer.

Changed_Credit_Limit: Indicates if the credit limit has been changed.

Num_Credit_Inquiries: Number of credit inquiries made by the customer.

Credit_Mix: Mix of different types of credit accounts held by the customer.

Outstanding_Debt: Amount of outstanding debt.

Credit_Utilization_Ratio: Ratio of credit used to credit available.

Credit_History_Age: Age of credit history.

Payment_of_Min_Amount: Indicates if minimum payment amount is met.

Total_EMI_per_month: Total Equated Monthly Installment (EMI) paid by the customer.

Amount_invested_monthly: Amount invested monthly by the customer.

Payment_Behaviour: Payment behavior of the customer.

Monthly_Balance: Monthly balance in the account.

Credit_Score: Target variable - credit score of the customer.

### Import libraries

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import re
import datetime
from sklearn.linear_model import LinearRegression
plt.style.use('ggplot')

%matplotlib inline
# %matplotlib notebook
plt.rcParams["figure.figsize"] = (12, 6)
# plt.rcParams['figure.dpi'] = 100
sns.set_style("whitegrid")
import warnings

warnings.filterwarnings("ignore")
warnings.warn("this will not show")
pd.set_option('display.float_format', lambda x: '%.3f' % x)


# Data cleaning

In [6]:
df = pd.read_csv("train.csv")
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,...,_,809.98,26.823,22 Years and 1 Months,No,49.575,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.945,NaN,No,49.575,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609,22 Years and 3 Months,No,49.575,81.69952126,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.378,22 Years and 4 Months,No,49.575,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,...,Good,809.98,24.797,22 Years and 5 Months,No,49.575,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [8]:
# Duplicate check
def duplicate_values(df):
    print("Duplicate check...")
    num_duplicates = df.duplicated(subset=None, keep='first').sum()
    if num_duplicates > 0:
        print("There are", num_duplicates, "duplicated observations in the dataset.")
        df.drop_duplicates(keep='first', inplace=True)
        print(num_duplicates, "duplicates were dropped!")
        print("No more duplicate rows!")
    else:
        print("There are no duplicated observations in the dataset.")
        
duplicate_values(df)

Duplicate check...
There are no duplicated observations in the dataset.


In [9]:
# dropping unnecessary columns for modelling
df = df.drop(['ID','Customer_ID','Month','Name','SSN', 'Type_of_Loan', 'Changed_Credit_Limit', 'Monthly_Inhand_Salary'], axis = 1)

In [10]:
target_mapping = {'Poor': 0, 'Standard': 1, 'Good': 2}

# Map the target variable using the defined mapping
df['Credit_Score'] = df['Credit_Score'].map(target_mapping)

In [11]:
df = df.replace("_", "", regex=True)
df[df.eq('').any(axis=1)] 

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,3,4,3,4,3,7,4.000,,809.98,26.823,22 Years and 1 Months,No,49.575,80.41529543900253,HighspentSmallvaluepayments,312.49408867943663,2
8,28,,34847.84,2,4,6,1,3,4,2.000,Good,605.03,24.464,26 Years and 7 Months,No,18.816,104.2918252,LowspentSmallvaluepayments,470.69062692529184,1
10,28,Teacher,34847.84,2,1385,6,1,3,-1,2.000,,605.03,33.225,26 Years and 9 Months,No,18.816,58.51597569589465,HighspentLargevaluepayments,466.46647639764313,1
16,34,,143162.64,1,5,8,3,5,8,3.000,Good,1303.01,28.617,17 Years and 9 Months,No,246.992,168.4137027,!@9#%8,1043.3159778669492,2
18,34,,143162.64,1,5,8,3,8,7,NaN,Good,1303.01,26.520,17 Years and 11 Months,No,246.992,10000,HighspentSmallvaluepayments,715.7413674,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99989,28,,20002.88,10,8,29,5,33,25,9.000,Bad,3571.7,33.360,6 Years and 1 Months,Yes,60.965,74.36660309585342,!@9#%8,307.659,1
99992,24,Mechanic,39628.99,4,6,7,2,23,NaN,3.000,,502.38,32.991,31 Years and 3 Months,No,35.104,401.1964806036356,LowspentSmallvaluepayments,189.641,0
99994,25,Mechanic,39628.99,4,6,7,2,20,6,3.000,,502.38,39.324,31 Years and 5 Months,No,35.104,140.58140274528395,HighspentMediumvaluepayments,410.256,0
99995,25,Mechanic,39628.99,4,6,7,2,23,7,3.000,,502.38,34.664,31 Years and 6 Months,No,35.104,60.97133255718485,HighspentLargevaluepayments,479.866,0


In [12]:
rows_with_empty_values = df[df.eq('').any(axis=1)]

# Fill empty values with np.nan in selected rows
df.loc[rows_with_empty_values.index] = df.loc[rows_with_empty_values.index].replace('', np.nan)

In [13]:
df.head()

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,3,4,3,4,3,7,4.000,NaN,809.98,26.823,22 Years and 1 Months,No,49.575,80.41529543900253,HighspentSmallvaluepayments,312.49408867943663,2
1,23,Scientist,19114.12,3,4,3,4,-1,NaN,4.000,Good,809.98,31.945,NaN,No,49.575,118.28022162236736,LowspentLargevaluepayments,284.62916249607184,2
2,-500,Scientist,19114.12,3,4,3,4,3,7,4.000,Good,809.98,28.609,22 Years and 3 Months,No,49.575,81.69952126,LowspentMediumvaluepayments,331.2098628537912,2
3,23,Scientist,19114.12,3,4,3,4,5,4,4.000,Good,809.98,31.378,22 Years and 4 Months,No,49.575,199.4580743910713,LowspentSmallvaluepayments,223.45130972736786,2
4,23,Scientist,19114.12,3,4,3,4,6,NaN,4.000,Good,809.98,24.797,22 Years and 5 Months,No,49.575,41.420153086217326,HighspentMediumvaluepayments,341.48923103222177,2


In [14]:
df.shape

(100000, 20)

In [15]:
df.isna().sum()

Age                             0
Occupation                   7062
Annual_Income                   0
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Num_Credit_Inquiries         1965
Credit_Mix                  20195
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Num_Bank_Accounts,100000.000,17.091,117.405,-1.000,3.000,6.000,7.000,1798.000
Num_Credit_Card,100000.000,22.474,129.057,0.000,4.000,5.000,7.000,1499.000
Interest_Rate,100000.000,72.466,466.423,1.000,8.000,13.000,20.000,5797.000
Delay_from_due_date,100000.000,21.069,14.860,-5.000,10.000,18.000,28.000,67.000
Num_Credit_Inquiries,98035.000,27.754,193.177,0.000,3.000,6.000,9.000,2597.000
Credit_Utilization_Ratio,100000.000,32.285,5.117,20.000,28.053,32.306,36.497,50.000
Total_EMI_per_month,100000.000,1403.118,8306.041,0.000,30.307,69.249,161.224,82331.000
Credit_Score,100000.000,0.888,0.675,0.000,0.000,1.000,1.000,2.000


In [17]:
df.describe(include='object').T

,count,unique,top,freq
Age,100000,1728,38,2994
Occupation,92938,15,Lawyer,6575
Annual_Income,100000,13575,109945.32,16
Num_of_Loan,100000,414,3,15104
Num_of_Delayed_Payment,92998,711,19,5481
Credit_Mix,79805,3,Standard,36479
Outstanding_Debt,100000,12209,1360.45,24
Credit_History_Age,90970,404,15 Years and 11 Months,446
Payment_of_Min_Amount,100000,3,Yes,52326
Amount_invested_monthly,95521,91049,10000,4305


### Find and replace values

In [19]:
df.Age.nunique()

1728

In [20]:
df.Age.value_counts(ascending=False).head(10)

Age
38    2994
28    2968
31    2955
26    2945
32    2884
36    2868
35    2866
25    2861
27    2859
39    2846
Name: count, dtype: int64

In [21]:
def non_numeric_values(df, column_name):
    # Define a regular expression pattern to match numeric characters
    pattern = r'\D+'
    
    # Find non-numeric values using regular expression
    non_numeric_values = df[column_name].astype(str).str.findall(pattern)
    
    # Flatten the list of lists
    non_numeric_values = [item for sublist in non_numeric_values for item in sublist]
    
    # Use set to ensure uniqueness
    unique_non_numeric_values = set(non_numeric_values)
    
    return unique_non_numeric_values

unique_non_numeric = non_numeric_values(df, 'Age')
print(unique_non_numeric)


{'-'}


In [22]:
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df[df['Age'] < 0]['Age'].unique()

array([-500])

In [23]:
df[df['Age'] > 75]['Age'].unique()

array([7580,  181,  995, ..., 4808, 2263, 1342])

In [24]:
df.drop(df[df['Age'] > 100].index, inplace = True)
df.drop(df[df['Age'] < 0].index, inplace = True)

In [25]:
df.shape

(97224, 20)

In [26]:
df.Occupation.value_counts()

Occupation
Lawyer          6389
Architect       6175
Engineer        6172
Scientist       6113
Accountant      6110
Mechanic        6099
MediaManager    6068
Teacher         6049
Developer       6031
Entrepreneur    6015
Doctor          5916
Journalist      5906
Manager         5807
Musician        5760
Writer          5741
Name: count, dtype: int64

In [27]:
df.Occupation.isna().sum()

6873

In [28]:
df.loc[pd.isna(df['Occupation']), 'Occupation'] = 'Other'

In [29]:
non_numeric_values(df, 'Num_Bank_Accounts')

{'-'}

In [30]:
df[df.Num_Bank_Accounts < 0]['Num_Bank_Accounts'].count()

21

In [31]:
df.drop(df[df['Num_Bank_Accounts'] < 0].index, inplace=True)

In [32]:
non_numeric_values(df, 'Num_of_Loan')

{'-'}

In [33]:
df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce')
df[df['Num_of_Loan'] < 0]['Num_of_Loan'].unique()

array([-100])

In [34]:
df['Num_of_Loan'].unique()

array([   4,    1,    3,  967, -100,    0,    2,    7,    5,    6,    8,
          9, 1464,  622,  352, 1017,  945,  146,  563,  341,  720, 1485,
         49,  737, 1106,  466,  728,  313,  597,  119,  663,  640,   92,
       1019,  501, 1302,   39,  716,  848,  931, 1214,  186,  424, 1001,
       1110, 1152,  457, 1433, 1187,   52, 1480, 1047, 1035, 1347,   33,
        193,  699,  329, 1451,  484,  649,  995,  545,  684, 1135, 1094,
       1204,  654,   58,  348,  614, 1363,  323, 1406, 1348,  430,  153,
       1461,  905, 1312, 1424, 1154,   95, 1353, 1228,  819, 1006,  795,
        359, 1209,  590,  696, 1185, 1465,  911,   70,  816, 1369,  143,
       1416,  455,   55, 1096, 1474,  420,  904,   89, 1259,  527, 1241,
        449,  983,  418,  319,   23,  238,  638,  138,  235,  280, 1070,
       1484,  274,  494, 1459,  404, 1354, 1495, 1391,  601, 1313, 1319,
        898,  231,  752,  174,  961, 1046,  834,  284,  438,  288, 1463,
       1151,  719,  198, 1015,  855,  841,  392, 14

In [35]:
df[df['Num_of_Loan'] < 0]['Num_of_Loan'].count() # cannot be negative, dropping them

3770

In [36]:
df.drop(df[df['Num_of_Loan'] < 0].index, inplace=True)
df.shape

(93433, 20)

In [37]:
non_numeric_values(df, 'Delay_from_due_date')

{'-'}

In [38]:
df[df['Delay_from_due_date'] < 0]['Delay_from_due_date'].unique()  # not possible

array([-1, -2, -3, -5, -4])

In [39]:
df['Num_of_Loan'].nunique()

398

In [40]:
df[df['Delay_from_due_date'] < 0]['Delay_from_due_date'].count()

547

In [41]:
df.drop(df[df['Delay_from_due_date'] < 0].index, inplace=True)
df.shape

(92886, 20)

In [42]:
non_numeric_values(df, 'Num_of_Delayed_Payment')

{'-', 'nan'}

In [43]:
df['Num_of_Delayed_Payment'] = pd.to_numeric(df['Num_of_Delayed_Payment'], errors='coerce')
df[df['Num_of_Delayed_Payment'] < 0]['Num_of_Delayed_Payment'].unique()

array([-1., -3., -2.])

In [44]:
df[df['Num_of_Delayed_Payment'] < 0]['Num_of_Delayed_Payment'].count()

592

In [45]:
df.drop(df[df['Num_of_Delayed_Payment'] < 0].index, inplace=True)
df.shape

(92294, 20)

In [46]:
df['Num_of_Delayed_Payment'].value_counts().head(10)

Num_of_Delayed_Payment
17.000    5080
19.000    5065
16.000    4951
10.000    4941
15.000    4888
18.000    4876
20.000    4761
12.000    4717
9.000     4617
8.000     4518
Name: count, dtype: int64

In [47]:
def knn_impute_column(df, column_name, n_neighbors=5):
    """
    Fill missing values in a specific column using KNN imputation.

    Parameters:
    df (DataFrame): The DataFrame containing the column to be imputed.
    column_name (str): The name of the column to be imputed.
    n_neighbors (int): Number of neighbors to consider for imputation. Default is 5.

    Returns:
    DataFrame: The DataFrame with missing values in the specified column imputed.
    """

    # Extract the column to be imputed
    column_to_impute = df[[column_name]]

    # Create an instance of the KNNImputer class
    imputer = KNNImputer(n_neighbors=n_neighbors)

    # Fit and transform the imputer to fill missing values in the column
    column_imputed = imputer.fit_transform(column_to_impute)

    # Replace the original column with the imputed values
    df[column_name] = column_imputed

    return df

In [48]:
df = knn_impute_column(df, 'Num_of_Delayed_Payment')

# to specify the number of neighbors:
# df = knn_impute_column(df, 'Num_of_Delayed_Payment', n_neighbors=10)

In [49]:
df['Num_of_Delayed_Payment'].isna().sum()

0

In [50]:
non_numeric_values(df, 'Num_Credit_Inquiries')

{'.', 'nan'}

In [51]:
df[df['Num_Credit_Inquiries'] < 0]['Num_Credit_Inquiries'].unique()

array([], dtype=float64)

In [52]:
df['Num_Credit_Inquiries'].isna().sum()

1810

In [53]:
df['Num_Credit_Inquiries'].nunique()

1160

In [54]:
df['Num_Credit_Inquiries'].value_counts().head(10)

Num_Credit_Inquiries
4.000    10314
3.000     8103
6.000     7560
7.000     7516
8.000     7311
2.000     7290
1.000     6889
0.000     6364
5.000     5277
9.000     4932
Name: count, dtype: int64

In [55]:
df = knn_impute_column(df, 'Num_Credit_Inquiries')
df['Num_Credit_Inquiries'].isna().sum()

0

In [56]:
df.Credit_Mix.value_counts()

Credit_Mix
Standard    34131
Good        21741
Bad         17755
Name: count, dtype: int64

In [57]:
df.Credit_Mix.isna().sum()

18667

In [58]:
# Fill null values in 'Credit_Mix' with 'Unknown'
df['Credit_Mix'].fillna('Unknown', inplace=True)

df['Credit_Mix'].astype('object')

0        Unknown
3           Good
4           Good
5           Good
6           Good
          ...   
99995    Unknown
99996    Unknown
99997       Good
99998       Good
99999       Good
Name: Credit_Mix, Length: 92294, dtype: object

In [59]:
df.Credit_Mix.value_counts()

Credit_Mix
Standard    34131
Good        21741
Unknown     18667
Bad         17755
Name: count, dtype: int64

In [60]:
df['Credit_History_Age'].value_counts()

Credit_History_Age
15 Years and 11 Months    414
19 Years and 5 Months     408
19 Years and 3 Months     407
17 Years and 9 Months     407
15 Years and 9 Months     406
                         ... 
0 Years and 3 Months       19
33 Years and 7 Months      14
0 Years and 2 Months       13
33 Years and 8 Months      10
0 Years and 1 Months        1
Name: count, Length: 404, dtype: int64

In [61]:
df['Credit_History_Age'].isna().sum()

8356

In [62]:
# Group by 'Credit_Mix' and calculate mode for 'Credit_History_Age' within each group
mode_by_credit_mix = df.groupby('Credit_Mix')['Credit_History_Age'].transform(lambda x: x.mode()[0])

# Fill NaN values in 'Credit_History_Age' with the mode for each group
df['Credit_History_Age'].fillna(mode_by_credit_mix, inplace=True)

In [63]:
df['Credit_History_Age'].isna().sum()

0

In [64]:
df['Credit_History_Age'].value_counts()

Credit_History_Age
19 Years and 5 Months     3525
30 Years and 2 Months     2203
15 Years and 9 Months     2102
8 Years and 10 Months     1828
15 Years and 11 Months     414
                          ... 
0 Years and 3 Months        19
33 Years and 7 Months       14
0 Years and 2 Months        13
33 Years and 8 Months       10
0 Years and 1 Months         1
Name: count, Length: 404, dtype: int64

In [65]:
# Extract years and months from 'Credit_History_Age' column
df['Credit_History_Years'] = df['Credit_History_Age'].str.extract(r'(\d+) Years').astype(int)
df['Credit_History_Months'] = df['Credit_History_Age'].str.extract(r'(\d+) Months').astype(int)

df.drop(columns=['Credit_History_Age'], inplace=True)

df.head()

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,...,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Credit_History_Years,Credit_History_Months
0,23,Scientist,19114.12,3,4,3,4,3,7.000,4.000,...,809.98,26.823,No,49.575,80.41529543900253,HighspentSmallvaluepayments,312.49408867943663,2,22,1
3,23,Scientist,19114.12,3,4,3,4,5,4.000,4.000,...,809.98,31.378,No,49.575,199.4580743910713,LowspentSmallvaluepayments,223.45130972736786,2,22,4
4,23,Scientist,19114.12,3,4,3,4,6,31.203,4.000,...,809.98,24.797,No,49.575,41.420153086217326,HighspentMediumvaluepayments,341.48923103222177,2,22,5
5,23,Scientist,19114.12,3,4,3,4,8,4.000,4.000,...,809.98,27.262,No,49.575,62.430172331195294,!@9#%8,340.4792117872438,2,22,6
6,23,Scientist,19114.12,3,4,3,4,3,8.000,4.000,...,809.98,22.538,No,49.575,178.3440674122349,LowspentSmallvaluepayments,244.5653167062043,2,22,7


In [66]:
# Convert years to months and add with months to get total months
df['Credit_History_Age_Months'] = df['Credit_History_Years'] * 12 + df['Credit_History_Months']

# Drop the 'Credit_History_Years' and 'Credit_History_Months' columns
df.drop(columns=['Credit_History_Years', 'Credit_History_Months'], inplace=True)

df.head()

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Credit_History_Age_Months
0,23,Scientist,19114.12,3,4,3,4,3,7.000,4.000,Unknown,809.98,26.823,No,49.575,80.41529543900253,HighspentSmallvaluepayments,312.49408867943663,2,265
3,23,Scientist,19114.12,3,4,3,4,5,4.000,4.000,Good,809.98,31.378,No,49.575,199.4580743910713,LowspentSmallvaluepayments,223.45130972736786,2,268
4,23,Scientist,19114.12,3,4,3,4,6,31.203,4.000,Good,809.98,24.797,No,49.575,41.420153086217326,HighspentMediumvaluepayments,341.48923103222177,2,269
5,23,Scientist,19114.12,3,4,3,4,8,4.000,4.000,Good,809.98,27.262,No,49.575,62.430172331195294,!@9#%8,340.4792117872438,2,270
6,23,Scientist,19114.12,3,4,3,4,3,8.000,4.000,Good,809.98,22.538,No,49.575,178.3440674122349,LowspentSmallvaluepayments,244.5653167062043,2,271


In [67]:
df.Payment_of_Min_Amount.value_counts()

Payment_of_Min_Amount
Yes    48966
No     32262
NM     11066
Name: count, dtype: int64

In [68]:
df.Amount_invested_monthly.nunique()

84073

In [69]:
non_numeric_values(df, 'Amount_invested_monthly')

{'.', 'nan'}

In [70]:
df['Amount_invested_monthly'] = pd.to_numeric(df['Amount_invested_monthly'], errors='coerce')
df[df['Amount_invested_monthly'] < 0]['Amount_invested_monthly'].unique()

array([], dtype=float64)

In [71]:
df.Amount_invested_monthly.value_counts().head(15)

Amount_invested_monthly
10000.000    3941
0.000         154
80.415          1
82.081          1
145.409         1
96.327          1
157.643         1
89.570          1
51.802          1
98.026          1
145.016         1
129.361         1
119.764         1
323.629         1
108.036         1
Name: count, dtype: int64

In [72]:
df[df.Amount_invested_monthly.isna()].sample(15)

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Credit_History_Age_Months
70480,22,Doctor,14618.9,8,4,20,6,24,19.000,8.000,Unknown,408.9,38.600,Yes,43.602,NaN,HighspentMediumvaluepayments,286.6825979310693,1,214
78572,26,Entrepreneur,98703.99,2,6,8,3,10,5.000,4.000,Good,1386.11,41.135,NM,142.666,NaN,HighspentLargevaluepayments,845.2262390306828,1,347
18688,25,Developer,57055.95,4,4,9,4,13,15.000,5.000,Standard,470.01,37.338,Yes,174.286,NaN,HighspentSmallvaluepayments,452.5356968711969,1,93
64666,14,Manager,19556.16,7,7,33,5,34,16.000,9.000,Unknown,2850.96,23.619,Yes,65.797,NaN,HighspentMediumvaluepayments,307.2892978204384,1,58
71975,26,Developer,40782.69,2,6,7,2,19,8.000,3.000,Good,1233.24,25.708,No,39.549,NaN,HighspentLargevaluepayments,470.7394511187983,0,362
20517,34,Doctor,62925.64,7,5,21,2,22,18.000,10.000,Standard,2060.33,25.561,Yes,94.355,NaN,LowspentSmallvaluepayments,88.93798954511192,0,240
61832,39,Accountant,17153.69,6,8,22,3,58,22.000,59.000,Bad,1757.24,24.218,Yes,38.063,NaN,LowspentSmallvaluepayments,291.6095716225683,0,227
76615,43,Mechanic,15302.74,10,10,24,5,32,22.000,6.000,Bad,4620.45,27.004,Yes,28197.000,NaN,LowspentSmallvaluepayments,196.78687332408063,0,159
66693,44,Teacher,8787.9,8,5,18,5,8,2348.000,9.000,Unknown,234.47,30.241,Yes,23.825,NaN,HighspentLargevaluepayments,254.8708872627381,2,169
53506,29,Journalist,8514.715,10,7,22,6,21,16.000,8.000,Bad,4403.86,36.477,Yes,21.483,NaN,LowspentLargevaluepayments,276.06210572532706,1,107


In [73]:
df = knn_impute_column(df, 'Amount_invested_monthly')

In [74]:
df.Amount_invested_monthly.isna().sum()

0

In [75]:
df.Amount_invested_monthly.value_counts().head(10)

Amount_invested_monthly
633.430      4128
10000.000    3941
0.000         154
80.415          1
82.081          1
145.409         1
96.327          1
157.643         1
89.570          1
51.802          1
Name: count, dtype: int64

In [76]:
df[df['Amount_invested_monthly'] == 10000]['Amount_invested_monthly'].count()

3941

In [77]:
df.Payment_Behaviour.value_counts()

Payment_Behaviour
LowspentSmallvaluepayments      23661
HighspentMediumvaluepayments    16179
LowspentMediumvaluepayments     12777
HighspentLargevaluepayments     12569
HighspentSmallvaluepayments     10468
LowspentLargevaluepayments       9646
!@9#%8                           6994
Name: count, dtype: int64

In [78]:
# Replace '!@9#%8' with NaN
df['Payment_Behaviour'].replace('!@9#%8', pd.NA, inplace=True)

# Impute missing values with mode
mode_value = df['Payment_Behaviour'].mode()[0]
df['Payment_Behaviour'].fillna(mode_value, inplace=True)

In [79]:
df.Payment_Behaviour.value_counts()

Payment_Behaviour
LowspentSmallvaluepayments      30655
HighspentMediumvaluepayments    16179
LowspentMediumvaluepayments     12777
HighspentLargevaluepayments     12569
HighspentSmallvaluepayments     10468
LowspentLargevaluepayments       9646
Name: count, dtype: int64

In [80]:
df['Payment_Behaviour'] = df['Payment_Behaviour'].str.replace('Lowspent', 'Low_spent_')
df['Payment_Behaviour'] = df['Payment_Behaviour'].str.replace('Highspent', 'High_spent_')
df['Payment_Behaviour'] = df['Payment_Behaviour'].str.replace('Highspent', 'High_spent_')

df['Payment_Behaviour'] = df['Payment_Behaviour'].str.replace('Smallvalue', 'Small_value_')
df['Payment_Behaviour'] = df['Payment_Behaviour'].str.replace('Largevalue', 'Large_value_')
df['Payment_Behaviour'] = df['Payment_Behaviour'].str.replace('Mediumvalue', 'Medium_value_')

df['Payment_Behaviour'].value_counts()

Payment_Behaviour
Low_spent_Small_value_payments      30655
High_spent_Medium_value_payments    16179
Low_spent_Medium_value_payments     12777
High_spent_Large_value_payments     12569
High_spent_Small_value_payments     10468
Low_spent_Large_value_payments       9646
Name: count, dtype: int64

In [81]:
df.Monthly_Balance.isna().sum()

1123

In [82]:
non_numeric_values(df, 'Monthly_Balance')

{'-', '.', 'nan'}

In [83]:
df['Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')
df[df['Monthly_Balance'] < 0]['Monthly_Balance'].count()

9

In [84]:
df.drop(df[df['Monthly_Balance'] < 0].index, inplace = True)

In [85]:
df['Monthly_Balance'].value_counts().head(15)

Monthly_Balance
312.494    1
286.520    1
395.106    1
440.468    1
411.910    1
397.622    1
327.670    1
399.045    1
355.826    1
199.857    1
304.184    1
539.684    1
614.241    1
519.046    1
494.545    1
Name: count, dtype: int64

In [86]:
df['Monthly_Balance'].max()

1602.0405189622518

In [87]:
df['Monthly_Balance'].min()

0.007759665

In [88]:
df['Monthly_Balance'].mean()

401.10228551216187

In [89]:
df = knn_impute_column(df, 'Monthly_Balance')
df.Monthly_Balance.isna().sum()

0

### Take Care of outliers

In [91]:
df['Num_Bank_Accounts'] = df['Num_Bank_Accounts'].apply(lambda x: 11 if x > 11 else x)

df['Num_Credit_Card'] = df['Num_Credit_Card'].apply(lambda x: 11 if x > 11 else x)

df['Num_of_Loan'] = df['Num_of_Loan'].apply(lambda x: 9 if x > 9 else x)

df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].apply(lambda x: 50 if x > 50 else x)

df['Num_Credit_Inquiries'] = df['Num_Credit_Inquiries'].apply(lambda x: 17 if x > 17 else x)

df['Interest_Rate'] = df['Interest_Rate'].apply(lambda x: 24 if x > 24 else x)

In [92]:
# Create the Score_Category column based on the value of Credit_Score
df['Score_Category'] = df['Credit_Score'].map({2: 'Good', 1: 'Standard', 0: 'Poor'})

# lastly change wrong dtypes..
df['Annual_Income'] = df['Annual_Income'].astype(float)
df['Outstanding_Debt'] = df['Outstanding_Debt'].astype(float)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92285 entries, 0 to 99999
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        92285 non-null  int64  
 1   Occupation                 92285 non-null  object 
 2   Annual_Income              92285 non-null  float64
 3   Num_Bank_Accounts          92285 non-null  int64  
 4   Num_Credit_Card            92285 non-null  int64  
 5   Interest_Rate              92285 non-null  int64  
 6   Num_of_Loan                92285 non-null  int64  
 7   Delay_from_due_date        92285 non-null  int64  
 8   Num_of_Delayed_Payment     92285 non-null  float64
 9   Num_Credit_Inquiries       92285 non-null  float64
 10  Credit_Mix                 92285 non-null  object 
 11  Outstanding_Debt           92285 non-null  float64
 12  Credit_Utilization_Ratio   92285 non-null  float64
 13  Payment_of_Min_Amount      92285 non-null  object 


In [93]:
df.to_csv("train_cleaned.csv", index=False)

In [94]:
df = pd.read_csv('train_cleaned.csv')
df.head()

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,...,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Credit_History_Age_Months,Score_Category
0,23,Scientist,19114.120,3,4,3,4,3,7.000,4.000,...,809.980,26.823,No,49.575,80.415,High_spent_Small_value_payments,312.494,2,265,Good
1,23,Scientist,19114.120,3,4,3,4,5,4.000,4.000,...,809.980,31.378,No,49.575,199.458,Low_spent_Small_value_payments,223.451,2,268,Good
2,23,Scientist,19114.120,3,4,3,4,6,31.203,4.000,...,809.980,24.797,No,49.575,41.420,High_spent_Medium_value_payments,341.489,2,269,Good
3,23,Scientist,19114.120,3,4,3,4,8,4.000,4.000,...,809.980,27.262,No,49.575,62.430,Low_spent_Small_value_payments,340.479,2,270,Good
4,23,Scientist,19114.120,3,4,3,4,3,8.000,4.000,...,809.980,22.538,No,49.575,178.344,Low_spent_Small_value_payments,244.565,2,271,Good
